## 0. Pre-given

In [ ]:
!pip install -U "datasets==3.6.0"

In [ ]:
### YOU MUST NOT CHANGE THIS CELL! ###

from datasets import load_dataset

full_dataset = load_dataset("skeskinen/TinyStories-GPT4", split="train")
full_dataset = full_dataset.remove_columns([c for c in full_dataset.column_names if c not in ["story", "features"]])
assert len(full_dataset) == 2745100

splits = full_dataset.train_test_split(test_size=10000, seed=42, shuffle=True)

train_dataset = splits["train"]
test_dataset  = splits["test"]

assert len(train_dataset) == 2735100
assert len(test_dataset)  == 10000

assert train_dataset[0]["story"][:33] == "One day, a little girl named Lily"
assert train_dataset[0]["features"] == ["Dialogue", "Conflict"]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/554 [00:00<?, ?B/s]

(…)-00000-of-00008-c63ccd5d5290f4a1.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00001-of-00008-478199d8ac044910.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00002-of-00008-9b868f59be94d815.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

...

(…)-00004-of-00008-5f8ac0bb66de5834.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00005-of-00008-e8c22c3e776b87dd.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00006-of-00008-941f57106aca3340.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00007-of-00008-771d8aa2d5ce5c95.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2745100 [00:00<?, ? examples/s]

In [ ]:
# Here we import all we need

from datasets import load_dataset, DatasetDict, load_from_disk

import torch

from google.colab import drive
drive.mount('/content/drive')

!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import torch.nn as nn

from torch.utils.data import DataLoader


Mounted at /content/drive


In [ ]:
# Here we print the first example of the train dataset

from pprint import pprint
pprint(train_dataset[0])

{'features': [],
 'labels': [0, 0, 0, 0, 0, 0],
 'story': 'Once upon a time, there was a sweet little girl named Lily. She '
          'loved to help her mom in the kitchen. One day, mom asked Lily to '
          'help her make dinner. Lily was very happy to help.\n'
          'Mom gave Lily a round thing called an onion. She asked Lily to peel '
          'the onion. As Lily peeled it, she started to cry. She did not know '
          'why. Mom saw Lily and gave her a hug.\n'
          'Then, mom helped Lily realize that the onion made her cry. But it '
          'was okay, because the onion would make their dinner taste very '
          'good. Lily felt better and finished helping mom. They had a yummy '
          'dinner and Lily learned something new.'}


## 1. Preprocessing

1.1 Train and validation split

In [ ]:
train_val_split = train_dataset.train_test_split(test_size=0.05, seed=42)  # 5% validation
train_dataset = train_val_split["train"]
val_dataset   = train_val_split["test"]

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")

Train size: 2598345
Validation size: 136755


1.2 Encode labels into binary vectors

In [ ]:
from datasets import load_dataset, DatasetDict


TAGS = ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]

def encode_labels(example):
    tag_set = set(example["features"])
    example["labels"] = [1 if tag in tag_set else 0 for tag in TAGS]
    return example


In [ ]:
# Fix: make sure all labels are torch.FloatTensor with correct shape
def format_labels(example):
    example["labels"] = torch.tensor(example["labels"], dtype=torch.float)
    return example

In [ ]:
#Map to all datasets
train_dataset = train_dataset.map(encode_labels)
val_dataset = val_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

Map:   0%|          | 0/2598345 [00:00<?, ? examples/s]

Map:   0%|          | 0/136755 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#labels need to be formatted
import torch

train_dataset = train_dataset.map(format_labels)
val_dataset = val_dataset.map(format_labels)
test_dataset = test_dataset.map(format_labels)

Map:   0%|          | 0/2598345 [00:00<?, ? examples/s]

Map:   0%|          | 0/136755 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#Saving the Dataset with binary labels

# Mount and delete old dataset directories
from google.colab import drive
drive.mount('/content/drive')

#!rm -rf "/content/drive/MyDrive/Final_Project/train_dataset"
#!rm -rf "/content/drive/MyDrive/Final_Project/val_dataset"
#!rm -rf "/content/drive/MyDrive/Final_Project/test_dataset"

#print("Existing dataset directories deleted from Drive.")

# Just save the datasets directly
train_dataset.save_to_disk("/content/drive/MyDrive/Final_Project/train_dataset")
val_dataset.save_to_disk("/content/drive/MyDrive/Final_Project/val_dataset")
test_dataset.save_to_disk("/content/drive/MyDrive/Final_Project/test_dataset")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/5 shards):   0%|          | 0/2598345 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/136755 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
#Loading the Dataset from Drive

from google.colab import drive
drive.mount('/content/drive')

from datasets import load_from_disk

train_dataset = load_from_disk("/content/drive/MyDrive/Final_Project/train_dataset")
val_dataset = load_from_disk("/content/drive/MyDrive/Final_Project/val_dataset")
test_dataset = load_from_disk("/content/drive/MyDrive/Final_Project/test_dataset")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_dataset[0]

{'story': 'Once upon a time, there was a sweet little girl named Lily. She loved to help her mom in the kitchen. One day, mom asked Lily to help her make dinner. Lily was very happy to help.\nMom gave Lily a round thing called an onion. She asked Lily to peel the onion. As Lily peeled it, she started to cry. She did not know why. Mom saw Lily and gave her a hug.\nThen, mom helped Lily realize that the onion made her cry. But it was okay, because the onion would make their dinner taste very good. Lily felt better and finished helping mom. They had a yummy dinner and Lily learned something new.',
 'features': [],
 'labels': [0, 0, 0, 0, 0, 0]}

1.3 Tokenizer

In [ ]:
#We are using a pretrained tokenizer distilbert-base-uncased from Hugging Face

!pip install transformers

from transformers import AutoTokenizer

MODEL_NAME = "distilbert-base-uncased" #i should put a reference to this model
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["story"],
        padding="max_length",     # pad to fixed size
        truncation=True,          # truncate long stories
        max_length=256,           # 1/2 max token length for BERT models, shortened for faster performance
    )

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#Apply Tokenizer to Datasets
#We are using smaller samples right now for quicker iteration

train_sample = train_dataset.select(range(20000))
val_sample   = val_dataset.select(range(5000))
test_sample  = test_dataset.select(range(1000))

tokenized_train = train_sample.map(tokenize_function, batched=True)
tokenized_val   = val_sample.map(tokenize_function, batched=True)
tokenized_test  = test_sample.map(tokenize_function, batched=True)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_val.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#Save the Tokenized Datasets to Drive

from google.colab import drive
drive.mount('/content/drive')

tokenized_train.save_to_disk("/content/drive/MyDrive/Final_Project/tokenized_train")
tokenized_val.save_to_disk("/content/drive/MyDrive/Final_Project/tokenized_val")
tokenized_test.save_to_disk("/content/drive/MyDrive/Final_Project/tokenized_test")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
#Load

tokenized_train = load_from_disk("/content/drive/MyDrive/Final_Project/tokenized_train")
tokenized_val = load_from_disk("/content/drive/MyDrive/Final_Project/tokenized_val")
tokenized_test = load_from_disk("/content/drive/MyDrive/Final_Project/tokenized_test")

In [ ]:
#tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


In [ ]:
tokenized_train[0]

{'labels': tensor([0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([  101,  2320,  2588,  1037,  2051,  1010,  2045,  2001,  1037,  4086,
          2210,  2611,  2315,  7094,  1012,  2016,  3866,  2000,  2393,  2014,
          3566,  1999,  1996,  3829,  1012,  2028,  2154,  1010,  3566,  2356,
          7094,  2000,  2393,  2014,  2191,  4596,  1012,  7094,  2001,  2200,
          3407,  2000,  2393,  1012,  3566,  2435,  7094,  1037,  2461,  2518,
          2170,  2019, 20949,  1012,  2016,  2356,  7094,  2000, 14113,  1996,
         20949,  1012,  2004,  7094, 20956,  2009,  1010,  2016,  2318,  2000,
          5390,  1012,  2016,  2106,  2025,  2113,  2339,  1012,  3566,  2387,
          7094,  1998,  2435,  2014,  1037,  8549,  1012,  2059,  1010,  3566,
          3271,  7094,  5382,  2008,  1996, 20949,  2081,  2014,  5390,  1012,
          2021,  2009,  2001,  3100,  1010,  2138,  1996, 20949,  2052,  2191,
          2037,  4596,  5510,  2200,  2204,  1012,  7094,  2371,  2488,  1998,


## 2. Model

2.1 Define the model

In [ ]:
#here we use the AutoModel using DistilBERT

from transformers import AutoModelForSequenceClassification
import torch.nn as nn
import torch

MODEL_NAME = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(TAGS), problem_type="multi_label_classification")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#checking the output
batch = tokenized_train[0]
for k in ['input_ids', 'attention_mask', 'labels']:
    batch[k] = batch[k].unsqueeze(0)  # Add batch dimension

with torch.no_grad():
    outputs = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"])
    print(outputs.logits.shape)  # should be [1, 6]


torch.Size([1, 6])


## 3. Training

3.1 Data Preparation

In [ ]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16  # Can increase if you have more memory

train_loader = DataLoader(tokenized_train, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(tokenized_val, batch_size=BATCH_SIZE)
test_loader = DataLoader(tokenized_test, batch_size=BATCH_SIZE)

3.2 Model, Optimizer and Loss

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
import torch.nn as nn
from torch.optim import AdamW

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=6,
    problem_type="multi_label_classification"
)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.BCEWithLogitsLoss()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3.3 Sample implementation of F1 score, pre-given

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

TAGS = ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]

def f1_per_tag(true_tag_lists, pred_tag_lists):
    """
    F1 for each of the six tags.

    Parameters
    ----------
    true_tag_lists : list[list[str]]
        Ground-truth tags for every example.
    pred_tag_lists : list[list[str]]
        Tags predicted by the model for every example.
    """
    tag_to_idx = {t: i for i, t in enumerate(TAGS)}
    n = len(true_tag_lists)
    assert len(true_tag_lists) == len(pred_tag_lists)

    # build binary indicator matrices
    y_true = np.zeros((n, len(TAGS)), dtype=int)
    y_pred = np.zeros_like(y_true)

    for i, tags in enumerate(true_tag_lists):
        for t in tags:
            if t in tag_to_idx:
                y_true[i, tag_to_idx[t]] = 1

    for i, tags in enumerate(pred_tag_lists):
        for t in tags:
            if t in tag_to_idx:
                y_pred[i, tag_to_idx[t]] = 1

    scores = f1_score(y_true, y_pred, average=None, zero_division=0)
    return {t: float(s) for t, s in zip(TAGS, scores)}

3.4 Training & Validation Loops

In [ ]:
from tqdm import tqdm
import torch
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Automatically use GPU if available
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Constants
accumulation_steps = 4
TAGS = ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]

# Model initialization
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(TAGS))
model.to(device)

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def train_epoch(model, loader, optimizer, loss_fn, device, accumulation_steps=4):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for i, batch in enumerate(tqdm(loader, desc="Training")):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device).float()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        loss = loss_fn(logits, labels)

        # Scale loss by accumulation steps to average over batches
        #simulate a larger batch size with gradient accumulation. For example, accumulate gradients every 4 steps
        loss = loss / accumulation_steps
        loss.backward()
        total_loss += loss.item()

        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(loader):
            optimizer.step()
            optimizer.zero_grad()

    return total_loss / len(loader)



def evaluate(model, loader, loss_fn, device, thresholds=None):
    model.eval()
    total_loss = 0
    all_true_tags = []
    all_pred_tags = []

    # Default thresholds
    if thresholds is None:
        thresholds = {tag: 0.5 for tag in TAGS}

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            batch = {k: v.to(device) for k, v in batch.items()}

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            loss = loss_fn(logits, labels)
            total_loss += loss.item()

            # Get predictions (sigmoid + threshold)
            probs = torch.sigmoid(logits).cpu().numpy()
            #preds = (probs > 0.5).astype(int)

            # Apply per-tag thresholds to get tag names
            for label_row, prob_row in zip(labels.cpu().numpy(), probs):
                true_tags = [TAGS[i] for i, val in enumerate(label_row) if val == 1]
                pred_tags = [TAGS[i] for i, val in enumerate(prob_row) if val >= thresholds[TAGS[i]]]
                all_true_tags.append(true_tags)
                all_pred_tags.append(pred_tags)

    f1_scores = f1_per_tag(all_true_tags, all_pred_tags)
    return total_loss / len(loader), f1_scores


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModel, AutoTokenizer
model_1 = AutoModel.from_pretrained("bert-base-uncased")
#model_1 = AutoModel.from_pretrained("/content/drive/MyDrive/Final_Project/my_model", local_files_only=True)
#tokenizer_1 = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Final_Project/my_model", local_files_only=True)

In [ ]:
model_1 = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Final_Project/my_model")
tokenizer_1 = DistilBertTokenizer.from_pretrained("/content/drive/MyDrive/Final_Project/my_model")


HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/content/drive/MyDrive/Final_Project/my_model'. Use `repo_type` argument if needed.

In [ ]:
model_1.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [ ]:
#first we build a function collect_val_outputs
#it runs the model once on the validation set
#it then stores predicted probabilites from sigmoid(logits) -> probs
#and binary truth labels (shape: [num_samples, num_tags]) -> true_labels

from sklearn.metrics import f1_score
import numpy as np
from tqdm import tqdm
import torch

def collect_val_outputs(model, loader, device):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Collecting validation outputs"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).float()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.sigmoid(logits)

            all_probs.append(probs.cpu())
            all_labels.append(labels.cpu())

    return torch.cat(all_probs, dim=0).numpy(), torch.cat(all_labels, dim=0).numpy()

#then we build a tune_thresholds function
#it tries out a range of threshs (0.1-0.9) for each tag and finds the one with highest F1

def tune_thresholds(probs, true_labels, tags, search_space=np.arange(0.1, 0.91, 0.05)):
    best_thresholds = {}

    for i, tag in enumerate(tags):
        best_f1 = 0
        best_thresh = 0.5
        for thresh in search_space:
            preds = (probs[:, i] >= thresh).astype(int)
            f1 = f1_score(true_labels[:, i], preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh
        best_thresholds[tag] = best_thresh

    return best_thresholds


In [ ]:
#thresholds = {
#    "BadEnding": 0.35,
#    "Conflict": 0.40,
#    "Dialogue": 0.50,
#    "Foreshadowing": 0.10,
 #   "MoralValue": 0.50,
#    "Twist": 0.60,
#}
import json
with open("/content/drive/MyDrive/Final_Project/thresholds.json", "r") as f:
    thresholds = json.load(f)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Final_Project/thresholds.json'

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Optimizer and scheduler
optimizer = torch.optim.AdamW(model_1.parameters(), lr=2e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=1, verbose=True)

TAGS = ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]
EPOCHS = 5

best_val_loss = float('inf')
patience = 3
patience_counter = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    # Train
    train_loss = train_epoch(model_1, train_loader, optimizer, loss_fn, device)

    # Validation
    val_probs, val_labels = collect_val_outputs(model_1, val_loader, device)
    thresholds = tune_thresholds(val_probs, val_labels, TAGS)
    val_loss, val_f1 = evaluate(model_1, val_loader, loss_fn, device, thresholds)

    # Adjust learning rate
    scheduler.step(val_loss)

    # Print metrics
    print(f"\nTrain Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print("F1 per tag:")
    for tag, score in val_f1.items():
        print(f"  {tag:15}: {score:.4f}")

    # Save best model
    min_delta = 1e-4
    if best_val_loss - val_loss > min_delta:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model_1.state_dict(), "best_model.pt")
        print("✓ Model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("Early stopping triggered.")
            break


NameError: name 'model_1' is not defined

In [ ]:
#train

optimizer = torch.optim.AdamW(model_1.parameters(), lr=2e-5)

TAGS = ["BadEnding", "Conflict", "Dialogue", "Foreshadowing", "MoralValue", "Twist"]
EPOCHS = 5

best_val_loss = float('inf')
patience = 3
patience_counter = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    # Train
    train_loss = train_epoch(model_1, train_loader, optimizer, loss_fn, device)

    # Collect validation outputs
    val_probs, val_labels = collect_val_outputs(model_1, val_loader, device)

    # Tune thresholds based on validation set
    thresholds = tune_thresholds(val_probs, val_labels, TAGS)

    print("\nTuned thresholds:")
    for tag, t in thresholds.items():
        print(f"  {tag:15}: {t:.2f}")

    # Evaluate with tuned thresholds
    val_loss, val_f1 = evaluate(model_1, val_loader, loss_fn, device, thresholds)

    print(f"\nTrain Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print("F1 per tag:")
    for tag, score in val_f1.items():
        print(f"  {tag:15}: {score:.4f}")



Epoch 1/10


Training: 100%|██████████| 1250/1250 [07:01<00:00,  2.97it/s]



Tuned thresholds:
  BadEnding      : 0.50
  Conflict       : 0.10
  Dialogue       : 0.40
  Foreshadowing  : 0.20
  MoralValue     : 0.75
  Twist          : 0.75


Evaluating: 100%|██████████| 63/63 [00:06<00:00,  9.02it/s]



Train Loss: 0.0064
Val Loss:   0.2443
F1 per tag:
  BadEnding      : 0.9450
  Conflict       : 0.4577
  Dialogue       : 0.9197
  Foreshadowing  : 0.5111
  MoralValue     : 0.8152
  Twist          : 0.8900

Epoch 2/10


...


Train Loss: 0.0021
Val Loss:   0.3208
F1 per tag:
  BadEnding      : 0.9488
  Conflict       : 0.4596
  Dialogue       : 0.9330
  Foreshadowing  : 0.4699
  MoralValue     : 0.8128
  Twist          : 0.8980

Epoch 10/10


Training: 100%|██████████| 1250/1250 [07:01<00:00,  2.96it/s]



Tuned thresholds:
  BadEnding      : 0.75
  Conflict       : 0.15
  Dialogue       : 0.20
  Foreshadowing  : 0.15
  MoralValue     : 0.65
  Twist          : 0.50


Evaluating: 100%|██████████| 63/63 [00:06<00:00,  9.01it/s]


Train Loss: 0.0015
Val Loss:   0.3370
F1 per tag:
  BadEnding      : 0.9327
  Conflict       : 0.4790
  Dialogue       : 0.9265
  Foreshadowing  : 0.4845
  MoralValue     : 0.8415
  Twist          : 0.9008


## 4. Saving the Model

In [ ]:
model_1.save_pretrained("/content/drive/MyDrive/Final_Project/my_model_1")
tokenizer_1.save_pretrained("/content/drive/MyDrive/Final_Project/my_model_1")

('/content/drive/MyDrive/Final_Project/my_model_1/tokenizer_config.json',
 '/content/drive/MyDrive/Final_Project/my_model_1/special_tokens_map.json',
 '/content/drive/MyDrive/Final_Project/my_model_1/vocab.txt',
 '/content/drive/MyDrive/Final_Project/my_model_1/added_tokens.json')

In [ ]:
import json

with open("/content/drive/MyDrive/Final_Project/thresholds.json", "w") as f:
    json.dump(thresholds, f)


In [ ]:
#Loading

#from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

#model_1 = DistilBertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Final_Project/my_model")
#tokenizer_1 = DistilBertTokenizer.from_pretrained("/content/drive/MyDrive/Final_Project/my_model")


## 6. Testing

In [ ]:
with open("/content/drive/MyDrive/Final_Project/thresholds.json", "r") as f:
    thresholds = json.load(f)


In [ ]:
# ipython-input-30-821b3f58e381

from torch.utils.data import DataLoader # Make sure DataLoader is imported if not already

BATCH_SIZE = 16 # Use the same BATCH_SIZE as training if desired

# test_dataset in the original code should be replaced with tokenized_test
test_loader = DataLoader(tokenized_test, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
thresholds = {
    "BadEnding": 0.65,
   "Conflict": 0.15,
   "Dialogue": 0.30,
    "Foreshadowing": 0.10,
    "MoralValue": 0.30,
    "Twist": 0.35,
}

val_loss, test_f1 = evaluate(model_1, val_loader, loss_fn, device, thresholds)

print(f"\nTest Loss: {val_loss:.4f}")
print("Test F1 per tag:")
for tag, score in test_f1.items():
    print(f"  {tag:15}: {score:.4f}")


Evaluating: 100%|██████████| 63/63 [00:07<00:00,  8.31it/s]


Test Loss: 0.2274
Test F1 per tag:
  BadEnding      : 0.9439
  Conflict       : 0.3860
  Dialogue       : 0.9250
  Foreshadowing  : 0.4509
  MoralValue     : 0.8324
  Twist          : 0.8766
